In [ ]:
 %load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
 
%%R
 
projDevClasses <- read.table("https://raw.githubusercontent.com/PGCC-Uefs/PGCC014/main/projDevClasses.csv?token=APT2VG36NSMDG3QPJN6J3JLAXFRQ6",header=TRUE, sep=",", na.strings="NA", dec=".", strip.white=TRUE)
projClassesSmells <- read.table("https://raw.githubusercontent.com/PGCC-Uefs/PGCC014/main/projClassesSmells.csv?token=APT2VGZF5DKEW6GP62GMSBLAXFRVO",header=TRUE, sep=",", na.strings="NA", dec=".", strip.white=TRUE)
 
########################################################################################################################
########## PARTE 1 - MANIPULA��O DE DADOS - CRIA��O DA TABELA COM COLUNA INDICANDO SE CLASSE � SMELL OU N�O
########################################################################################################################
 
# Cria coluna para indicar que a classe � smell
projDevClassesIndicacaoSmell <- projClassesSmells
projDevClassesIndicacaoSmell$ehSmell <- "true" 
 
# faz um left join entre as tabelas que tem as classes mexidas pelos desenvolvedores e as classes que tem smell
projDevClassesEhSmell <- merge(x=projDevClasses,y=projDevClassesIndicacaoSmell,by=c("project","class"),all.x=TRUE)
 
# troca campos que ficaram com valor nulo ap�s o left join
projDevClassesEhSmell <- replace(projDevClassesEhSmell, is.na(projDevClassesEhSmell), "false")
 
########################################################################################################################
########## PARTE 2 - VISUALIZA��O DO N�MERO DE CLASSES MEXIDAS PELOS DESENVOLVEDORES
########################################################################################################################
 
# Muda o nome da tabela
dados <- projDevClassesEhSmell
 
#Mudando o nome das colunas
colnames(dados)[1] <- "Project"
colnames(dados)[2] <- "Class"
colnames(dados)[3] <- "Dev"
colnames(dados)[4] <- "Smell"
 
 
# Biblioteca para manipular dados a partir de agrupamentos. Boa para contar frequ�ncias, por exemplo
library(plyr)
 
# Cria tabela com n�mero de classes mexidas por desenvolvedor
numeroClassesMexidasPorDesenvolvedor <- count(dados, 'Dev')
 
# Para visualizar de forma ordenada pela frequencia
numeroClassesMexidasPorDesenvolvedor <- numeroClassesMexidasPorDesenvolvedor[order(numeroClassesMexidasPorDesenvolvedor$freq),] 
 
# Distribui��o do n�mero de classes trabalhadas por desenvolvedor
boxplot(numeroClassesMexidasPorDesenvolvedor$freq) 
boxplot(numeroClassesMexidasPorDesenvolvedor$freq,outline=FALSE)
 
# Valores dos quartis
summary(numeroClassesMexidasPorDesenvolvedor$freq)
##   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
##   1.00    11.75   44.50   157.14   182.50  1465.00 
 
#Valores de outliers
outlier_values <- boxplot.stats(numeroClassesMexidasPorDesenvolvedor$freq)$out 
outlier_values
#[1] 506  522  591  864  990 1183 1465
 
########################################################################################################################
########## PARTE 3 - VISUALIZAR EM UM BARPLOT A RELA��O ENTRE CLASSES MEXIDAS PELOS DESENVOLVEDORES QUE S�O SMELLS E QUE N�O S�O SMELLS
########################################################################################################################
 
### Monta tabela com valores de classes mexidas com e sem smells - separadas
library("dplyr")
 
resumoSmells <- dados %>% 
  group_by(Dev, Smell) %>%
  summarise(number = n())
 
resumoSmells <- data.frame(resumoSmells)
 
## Descobre quem tem somente classes que n�o s�o smells
## o objetivo de descobrir esses desenvolvedores � para adicionar a linha do desenvolvedor com valor 0 para numero de classes com smell = true
soDesenvolvedores <-resumoSmells$Dev
soDesenvolvedores <- data.frame(soDesenvolvedores[!(duplicated(soDesenvolvedores)|duplicated(soDesenvolvedores, fromLast=TRUE))])
colnames(soDesenvolvedores)[1] <- "Dev"
 
## S� interesse a linha 99 - peterfriese
#            Dev Smell number
#3        alikat false      1
#36       dereed  true      1
#39     egiraudy  true      2
#42      galderz  true      4
#55   intensifly false      1
#78       loonix  true      2
#99  peterfriese false     21
#108     rameshm  true      8
#143       Yoriy  true      3
subset(resumoSmells, (resumoSmells$Dev %in% soDesenvolvedores$Dev))
 
## Confere os tipos das colunas, pois ao inserir o valor, mudou os tipos e devem retornar
table(sapply(resumoSmells, class))
 
# Adiciona o n�mero de smells = 0 para o desenvolvedor que s� mexeu em classes sem smells
resumoSmells[nrow(resumoSmells) + 1,] = c("peterfriese","true", 0)
# Volta os tipos das colunas aos originais
resumoSmells$Dev <- as.character(resumoSmells$Dev)
resumoSmells$Smell <- as.factor(resumoSmells$Smell)
resumoSmells$number <- as.integer(resumoSmells$number)
 
# s� para conferir
#subset(resumoSmells, resumoSmells$Dev == "peterfriese")
 
# Faz um join para adicionar o total de classes mexidas na tabela resumo smells (ser� usado na hora de separar em quartis)
#so pra teste, faz um subset para contar
#subset(numeroClassesMexidasPorDesenvolvedor, numeroClassesMexidasPorDesenvolvedor$Dev == "andyhot")
resumoSmells <- merge(x=resumoSmells,y=numeroClassesMexidasPorDesenvolvedor,by=c("Dev"),all.x=TRUE)
resumoSmells <- resumoSmells %>% 
  rename(
    classesMexidas = freq
    )
 
# Biblioteca com fun��es diversas para criar gr�ficos
library(ggplot2)
 
### Para facilitar a visualia��o, remover todos abaixo de 11.75 (primeiro quartil)
desenvolvedoresQueMexeramEmPoucosSmells <- subset(numeroClassesMexidasPorDesenvolvedor, freq <= 11.75)
 
#Filtra...s� deixa os desenvolvedeores que mexeram em muitos smells (>11.75)
mexeramEmMuitosSmells <- dados
mexeramEmMuitosSmells <- mexeramEmMuitosSmells[!mexeramEmMuitosSmells$Dev %in% desenvolvedoresQueMexeramEmPoucosSmells$Dev,]
 
resumoSmellsAlterado <- mexeramEmMuitosSmells %>% 
  group_by(Dev, Smell) %>%
  summarise(number = n())
 
resumoSmellsAlterado <- data.frame(resumoSmellsAlterado)
 
## Soma o n�mero total de classes mexidas por todos os desenvolvedores
# 11859
sum(resumoSmellsAlterado$number)
 
# Monta o barplot da rela��o classes mexidas X smells - sem os desenvolvedores que mexeram em poucas classes..s� para facilitar a a visualiza��o
ggplot(resumoSmellsAlterado, aes(x = reorder(factor(Dev), -number), number, fill = Smell)) +  geom_bar(stat="identity") 
 
########################################################################################################################
########## PARTE 4 - VERIFICA A CORRELA��O ENTRE A QUANTIDADE DE SMELLS E QUANTIDADE DE CLASSES MEXIDAS
########################################################################################################################
 
## Gera tamb�m gr�fico de dispers�o da quantidade de classes mexidas X quantidade de smells
resumoSmellsSoTrue <- subset(resumoSmells, Smell == "true")
## Remove aqueles que est�o abaixo do primeiro quartil (11.75) e gera o gr�fico de dispers�o de novo
resumoSmellsRelacaoTrueSemExtremos <- subset(resumoSmellsSoTrue, classesMexidas > 11)
 
plot(resumoSmellsRelacaoTrueSemExtremos$classesMexidas, resumoSmellsRelacaoTrueSemExtremos$number, xlab="Classes mexidas", ylab="smelles", pch=19) 
 
## Calcula a correla��o n�mero de classes com Smells X N�mero de classes mexidas
# Pearson: 9580555, p-value < 2.2e-16
cor.test(resumoSmellsRelacaoTrueSemExtremos$classesMexidas, resumoSmellsRelacaoTrueSemExtremos$number, method=c("pearson"))
# Spearman: 0.8732981, p-value = 2.2e-16
cor.test(resumoSmellsRelacaoTrueSemExtremos$classesMexidas, resumoSmellsRelacaoTrueSemExtremos$number, method=c("spearman"))
# Kendal: 0.740322, p-value = 5.568e-16
cor.test(resumoSmellsRelacaoTrueSemExtremos$classesMexidas, resumoSmellsRelacaoTrueSemExtremos$number, method=c("kendal"))
 
########################################################################################################################
########## PARTE 5 - ANALISE DO PERCENTUAL DE CLASSES MEXIDAS QUE S�O SMELLS
########################################################################################################################
 
# Coloca valores false e true como colunas
paraCalculoRelacaoTrueFalse<-resumoSmellsAlterado
paraCalculoRelacaoTrueFalse<-reshape(paraCalculoRelacaoTrueFalse,timevar="Smell",idvar="Dev",direction="wide")
 
# Transforma NA em 0
paraCalculoRelacaoTrueFalse[is.na(paraCalculoRelacaoTrueFalse)] <- 0
 
# Cria coluna para calcular rela��o classes mexidas true e false (para smells)
paraCalculoRelacaoTrueFalse <- transform(paraCalculoRelacaoTrueFalse, relacaoTrue= number.true*100/(number.true+number.false))
paraCalculoRelacaoTrueFalse <- transform(paraCalculoRelacaoTrueFalse, relacaoFalse= number.false*100/(number.true+number.false))
 
attach(paraCalculoRelacaoTrueFalse)
paraCalculoRelacaoTrueFalse <- paraCalculoRelacaoTrueFalse[order(relacaoTrue, number.true, Dev),]
 
# ATEN��O!!! GAMBIARRA...precisei inserir a linha de valor true para o desenvolvedor "peterfriese", pois na hora do c�lculo ele n�o tinha linha para as classes com smell pois o valor era 0
# Adiciona o n�mero de smells = 0 para o desenvolvedor que s� mexeu em classes sem smells
#resumoSmellsAlterado[nrow(resumoSmellsAlterado) + 1,] = c("peterfriese","true", 0)
 
# Volta os tipos das colunas aos originais??? precisa disso?
resumoSmellsAlterado$Dev <- as.character(resumoSmellsAlterado$Dev)
resumoSmellsAlterado$Smell <- as.factor(resumoSmellsAlterado$Smell)
resumoSmellsAlterado$number <- as.integer(resumoSmellsAlterado$number)
 
# Monta tabela final depois de inserir a rela��o smells true X smells false (o objetivo � ordenar pela rela��o)
resumoSmellsAlterado <- merge(resumoSmellsAlterado, paraCalculoRelacaoTrueFalse[, c("Dev", "relacaoTrue")], by="Dev")
resumoSmellsAlterado <- merge(resumoSmellsAlterado, paraCalculoRelacaoTrueFalse[, c("Dev", "relacaoFalse")], by="Dev")
 
### Desmonta em duas tabelas (com valores true e false em separado) para depois remontar os percentuais em uma s� coluna
resumoSmellsRelacaoTrue <- subset(resumoSmellsAlterado, resumoSmellsAlterado$Smell == "true")
resumoSmellsRelacaoTrue$relacaoFalse <- NULL
resumoSmellsRelacaoTrue <- resumoSmellsRelacaoTrue %>% 
  rename(
    relacao = relacaoTrue
    )
 
resumoSmellsRelacaoTrue <- resumoSmellsRelacaoTrue[order(-resumoSmellsRelacaoTrue$relacao),]
 
### Gr�fico de barras mostrando os percentuais pelos desenvolvedores
ggplot(resumoSmellsRelacaoTrue, aes(x = reorder(Dev, -relacao), y = relacao)) + 
  geom_bar(stat="identity") + theme(axis.text.x = element_text(angle = 90, hjust = 1)) + ylim(0, 100) + geom_hline(yintercept = 50)+ geom_hline(yintercept = 75)+ geom_hline(yintercept = 25)
 
# S� para testar
 
#resumoSmellsRelacaoTrue$Dev <- as.character(resumoSmellsRelacaoTrue$Dev)
#resumoSmellsRelacaoTrue <- resumoSmellsRelacaoTrue[order(-resumoSmellsRelacaoTrue$Dev),]
 
## Total de desenvolvedores
## 57
nrow(resumoSmellsRelacaoTrue)
 
## Total de desenvolvedores em que mais de 75% das classes mexidas s�o smells
## 7
nrow(subset(resumoSmellsRelacaoTrue, resumoSmellsRelacaoTrue$relacao >= 75))
 
## Total de desenvolvedores em que mais de 50% das classes mexidas s�o smells
## 32
nrow(subset(resumoSmellsRelacaoTrue, resumoSmellsRelacaoTrue$relacao >= 50))
 
## Total de desenvolvedores em que menos de 25% das classes mexidas s�o smells
## 8
nrow(subset(resumoSmellsRelacaoTrue, resumoSmellsRelacaoTrue$relacao <= 25))
 
### Boxplot mostrando a distribui��o dos percentuais de classesmexidas que s�o smells
## Comentado porque este boxplot n�o ajudou na an�lise
##boxplot(resumoSmellsRelacaoTrue$relacao) 
 
########################################################################################################################
########## PARTE 6 - ANALISA SE OS DESENVOLVEDORES TEM MAIOR PERCENTUAL DE SMELLS PORQUE MEXERAM EM MAIS CLASSES
########################################################################################################################
 
# Cria coluna com valor percentual de classes mexidas que s�o smells com rela��o ao total de classes mexidas
resumoSmellsRelacaoTrueSemExtremos <- transform(resumoSmellsRelacaoTrueSemExtremos, relacaoTrue= number*100/(classesMexidas))
 
#### Limiar 50%
# Primeiro consideramos um limiar de 50% grupos com mais ou menos percentual de smell
# Cria coluna com indica��o se o desenvolver est� no grupo de alto percentual de classes com smell ou no grupo de baixo
# Neste momento considera 2 grupos (com mais ou menos de 50% das classes mexidas sendo smells)
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue > 50] <- "Many"
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue<=50] <- "Few"
 
# Conta o n�mero de desenvolvedores que est�o nos grupos "many" e "few"
# 31
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "Many"))
# 26
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "Few"))
 
# Visualiza distribui��o do n�meo de classes mexidas X percentual de classes mexidas que s�o smells
boxplot(classesMexidas ~ grupoMuitoOuPoucosSmells, data = resumoSmellsRelacaoTrueSemExtremos, xlab = "Qtde classes mexidas que s�o smells",
   ylab = "Qtde classes mexidas")
 
#### Limiar 60%
# Para melhorar a distribui��o, sem perder o sentido do que queremos mostras (que � que n�o h� diferen�a significativa com rela��o ao n�mero de classes mexidas - o percentual de classes mexidas varia independente disso) consideramos um limiar de 60% grupos com mais ou menos percentual de smell
# Cria coluna com indica��o se o desenvolver est� no grupo de alto percentual de classes com smell ou no grupo de baixo
# Neste momento considera 2 grupos (com mais ou menos de 60% das classes mexidas sendo smells)
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue > 60] <- "Many"
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue<=60] <- "Few"
 
# Conta o n�mero de desenvolvedores que est�o nos grupos "many" e "few"
# 22
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "Many"))
# 35
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "Few"))
 
# Visualiza distribui��o do n�meo de classes mexidas X percentual de classes mexidas que s�o smells
boxplot(classesMexidas ~ grupoMuitoOuPoucosSmells, data = resumoSmellsRelacaoTrueSemExtremos, xlab = "Qtde classes mexidas que s�o smells",
   ylab = "Qtde classes mexidas")
 
###### Agora formaremos 4 grupos (at� 25%, 25-50%, 50-75%, 75-100%)
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue < 25] <- "A-Very Few"
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue>=25 & resumoSmellsRelacaoTrueSemExtremos$relacaoTrue<50] <- "B-Few"
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue>=50 & resumoSmellsRelacaoTrueSemExtremos$relacaoTrue<75] <- "C-Many"
resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells[resumoSmellsRelacaoTrueSemExtremos$relacaoTrue>=75] <- "D-Many Many"
 
# Conta o n�mero de desenvolvedores que est�o nos grupos "many many, "many", "few" e "very few"
# 8
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "A-Very Few"))
# 17
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "B-Few"))
#25
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "C-Many"))
#7
nrow(subset(resumoSmellsRelacaoTrueSemExtremos, resumoSmellsRelacaoTrueSemExtremos$grupoMuitoOuPoucosSmells == "D-Many Many"))
 
# Visualiza distribui��o do n�meo de classes mexidas X percentual de classes mexidas que s�o smells
boxplot(classesMexidas ~ grupoMuitoOuPoucosSmells, data = resumoSmellsRelacaoTrueSemExtremos, xlab = "Qtde classes mexidas que s�o smells",
   ylab = "Qtde classes mexidas")
 
 
#################
#OBSERVA��O - O N�MERO DE VEZES QUE A CLASSE FOI MEXIDA IMPACTA NELA SER SMELL OU N�O??? VER ARTIGO DE THALITA td
#EM QUE MOMENTO O CARA INSERIU SMELL???

R[write to console]: Error in file(file, "rt") : 
  cannot open the connection to 'https://raw.githubusercontent.com/PGCC-Uefs/PGCC014/main/projDevClasses.csv?token=APT2VG36NSMDG3QPJN6J3JLAXFRQ6'

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In file(file, "rt") :
R[write to console]: 
 
R[write to console]:  cannot open URL 'https://raw.githubusercontent.com/PGCC-Uefs/PGCC014/main/projDevClasses.csv?token=APT2VG36NSMDG3QPJN6J3JLAXFRQ6': HTTP status was '404 Not Found'




Error in file(file, "rt") : 
  cannot open the connection to 'https://raw.githubusercontent.com/PGCC-Uefs/PGCC014/main/projDevClasses.csv?token=APT2VG36NSMDG3QPJN6J3JLAXFRQ6'


RInterpreterError: ignored